- author: Lee Meng
- date: 2019-05-06 09:00
- title: 用 CartoonGAN 及 TensorFlow 2 畫動漫
- slug: generate-anime-using-cartoongan-and-tensorflow2
- tags: GAN, TensorFlow, TensorFlow.js
- description: 
- summary: 
- image: finan-akbar-429027-unsplash-transformed.jpg
- image_credit_url: https://unsplash.com/photos/hwLAI5lRhdM?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText
- status: draft

In [ ]:
#ignore
- 加入 CartoonGAN 訓練過程介紹
- 翻譯中文
- 測試 tfjs model (using constant pad)
- 將 mnicnc 新 train 出來的加入 model option
- 將 tfjs demo 改成中文版
- 改善 gallery style


In [ ]:
#ignore
下一個 markdown cell 引入 cartoogan 用的 css

<link rel="stylesheet" href="{static}tfjs-apps/cartoongan/cartoongan.css" />

In [ ]:
#ignore
CartoonGAN css 引入結束

!quote
- 如果能讓 AI 幫我們將圖片自動轉換成新海誠或是宮崎駿等人的動漫風格，豈不是很有意思？

這篇文章介紹我與夥伴 [mnicnc404](https://github.com/mnicnc404) 最近以 [TensorFlow 2.0 Alpha](https://www.tensorflow.org/alpha) 實作的 [CartoonGAN-TensorFlow2
](https://github.com/mnicnc404/CartoonGan-tensorflow) 專案。我們同時也展示一個 [TensorFlow.js](https://www.tensorflow.org/js) 應用，讓你可以直接在瀏覽器上產生動漫。

[CartoonGAN](http://openaccess.thecvf.com/content_cvpr_2018/papers/Chen_CartoonGAN_Generative_Adversarial_CVPR_2018_paper.pdf) 於 2018 [CVPR](http://cvpr2019.thecvf.com/) 推出，是一個嘗試將真實世界圖片轉換成動漫的[對抗生成網路（Generative Adversarial Network, GAN）](https://youtu.be/yFBFl1cLYx8?list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&t=1879)。

!mp4
- images/cartoongan/cat_cover.mp4
- images/cartoongan/cat_cover.jpg
- 左上為原圖，其餘三圖則為我們使用 CartoonGAN 將不同動漫風格套用到原圖上的結果

當初看到這篇[論文](http://openaccess.thecvf.com/content_cvpr_2018/papers/Chen_CartoonGAN_Generative_Adversarial_CVPR_2018_paper.pdf)覺得很有趣，且裡頭正好展示了我喜愛的兩位日本動畫作家：[新海誠](https://zh.wikipedia.org/wiki/%E6%96%B0%E6%B5%B7%E8%AA%A0)及[宮崎駿](https://zh.wikipedia.org/zh-tw/%E5%AE%AB%E5%B4%8E%E9%AA%8F)的風格轉換結果，因此決定寫個 [TensorFlow.js](https://www.tensorflow.org/js) 應用，讓更多人可以輕鬆地體驗這個有趣的 CartoonGAN。

使用方法很直覺，選擇風格並上傳照片，完成！

!mp4
- images/cartoongan/tfjs-demo.mp4

背後處理主要分為兩個步驟：
- 下載模型
- 轉換圖片

TensorFlow.js 完全在瀏覽器裡運行，此頁面背後並沒有任何伺服器。因此確保以下條件能讓你獲得較好的體驗：
- 使用桌電 / 筆電等**計算能力**較強的設備
- 在網速快的環境測試（減少**載入模型**時間）

## 動手玩 CartoonGAN

以下就是實際的應用：

In [ ]:
#ignore
cartoongan demo input / output 開始

<div id="container" class="row">
    <div class="column50">
        <div>
            <img id="input" class="cartoongan_image" src="{static}tfjs-apps/cartoongan/cat.png"/>
        </div>
    </div>
    <div class="column50">
        <div>
            <img id="pregenerated_output" class="cartoongan_image" src="{static}tfjs-apps/cartoongan/cat_shinkai.jpg"/>
            <canvas id="output" style="display:none"></canvas>
        </div> 
        <div id="app-status" style="display:none;padding:70px 0"></div>
    </div> 
</div>
<div class="row" style="margin-bottom:6rem;">
    <div class="column50">
        <label class="btn" style="margin-top: 2rem;height: 3rem;line-height:2.8rem;color:white;padding: 0 1rem">
            選擇圖片
            <input type="file" id="files" name="files[]" multiple="" style="display:none">
        </label>
    </div> 
    <div class="column50">
        <select id="styles" name="styles" style="margin: auto;display: block;margin-top: 0.8rem">
            <option value="shinkai" selected="selected"><b>新海誠</b>風格</option>
            <option value="hayao"><b>宮崎駿</b>風格</option>
            <option value="hosoda"><b>細田守</b>風格</option>
            <option value="paprika"><b>盜夢偵探</b>風格</option>
        </select>
    </div> 
</div>

In [ ]:
#ignore
cartoongan demo input / output 結束

如何？結果出來了嗎？

如果你的設備（如手機）在上傳圖片後一直停在：
- `Loading Models`（網路速度不夠快，下載耗時）
- `Cartoonizing images`（設備運算資源的不足）

可以嘗試使用下一節介紹的方法來做轉換。保證有效。

## TensorFlow 2 畫動漫

適逢 [PoweredByTF 2.0 挑戰](https://tensorflow.devpost.com/)，我們也用 TensorFlow 2.0 Alpha 實作了 CartoonGAN。

如果你想要轉換大圖或是動圖，可以執行[這個 Colab 筆記本](https://colab.research.google.com/drive/1WIZBHix_cYIGsBKa4phIwCq5qXwO8fRX)： 

!mp4
- images/cartoongan/colab-demo.mp4
- images/cartoongan/colab-demo.jpg
- 執行我們準備的 Colab 筆記本可以讓你用 CartoonGAN 轉換任何圖片

[Google Colaboratory](https://colab.research.google.com/) 是一個雲端 [Jupyter 筆記本](https://jupyter.org/)環境，提供 GPU 讓任何人都可以立即開始一個深度學習專案。

在[這個筆記本](https://colab.research.google.com/drive/1WIZBHix_cYIGsBKa4phIwCq5qXwO8fRX)裡頭，以下步驟都幫你寫好了：
- 建置 TensorFlow 2.0 環境
- 下載 Github Repo 及預訓練模型 
- 下載任意網路圖片 / gif
- 使用 CartoonGAN 轉換圖片

你只需打開筆記本並依照指示一步步執行，即可為你的圖片添加動漫風格。

## 一些轉換後的動漫結果

獨樂樂不如眾樂樂。這一節讓我和你分享一些我們用 CartoonGAN 得到的結果。

以下每張圖片都分為四個區塊，從左到右、由上而下分別為：
- 原始圖片
- [新海誠](https://zh.wikipedia.org/wiki/%E6%96%B0%E6%B5%B7%E8%AA%A0)風格
- [宮崎駿](https://zh.wikipedia.org/zh-tw/%E5%AE%AB%E5%B4%8E%E9%AA%8F)風格
- [細田守](https://zh.wikipedia.org/wiki/%E7%B4%B0%E7%94%B0%E5%AE%88)風格

你可以點擊下圖左右兩側的小箭頭來切換不同圖片：

In [ ]:
#ignore
Gallery 的 CSS / JS / HTML 

<style>
<!-- https://www.w3schools.com/w3css/w3css_slideshow.asp -->
.w3-content,
.w3-auto {
	margin-left: auto;
	margin-right: auto
}

.w3-content {
	max-width: 980px
}

.w3-display-container:hover .w3-display-hover {
	display: block
}

.w3-display-container:hover span.w3-display-hover {
	display: inline-block
}

.w3-display-container {
	position: relative
}

.w3-button:hover {
	color: #000!important;
	background-color: inherit;
}

.w3-button {
	border: none;
	display: inline-block;
	padding: 8px 16px;
	vertical-align: middle;
	overflow: hidden;
	text-decoration: none;
	color: inherit;
	background-color: inherit;
	text-align: center;
	cursor: pointer;
	white-space: nowrap
}

.w3-button {
	-webkit-touch-callout: none;
	-webkit-user-select: none;
	-khtml-user-select: none;
	-moz-user-select: none;
	-ms-user-select: none;
	user-select: none
}

.w3-button:disabled {
	cursor: not-allowed;
	opacity: 0.3
}

.w3-display-left {
	position: absolute;
	top: 50%;
	left: 0%;
	transform: translate(0%, -50%);
	-ms-transform: translate(-0%, -50%)
}

.w3-display-right {
	position: absolute;
	top: 50%;
	right: 0%;
	transform: translate(0%, -50%);
	-ms-transform: translate(0%, -50%)
}
    
.mySlides {display:none;}
</style>

<div class="w3-content w3-display-container">
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/dance.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  
  <!--marvel-->
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/iron-man-face.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/iron-man.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>

  <img class="mySlides" src="{static}images/cartoongan/gallery/static-woman-face.jpg" style="width:100%">
  <img class="mySlides" src="{static}images/cartoongan/gallery/static-temple.jpg" style="width:100%">
  
  <!--cat-->
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/cat-shake-meme.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/cat-being-poked.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/cat-computer.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  
  <!--scenary-->
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/big-ben.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/city-street.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/church.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  
  <!--idol-->
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/demo.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/arakaki.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/harry-potter.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  
  <!--virtual character-->
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/pikachu.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>
  <video class="mySlides" loop muted autoplay playsinline style="width:100%">
      <source src="{static}images/cartoongan/gallery/kumamon.mp4" type="video/mp4">
        您的瀏覽器不支援影片標籤，請留言通知我：S
  </video>

  <button class="w3-button w3-black w3-display-left" onclick="plusDivs(-1)">&#10094;</button>
  <button class="w3-button w3-black w3-display-right" onclick="plusDivs(1)">&#10095;</button>
</div>

<script>
var slideIndex = 1;
showDivs(slideIndex);

function plusDivs(n) {
  showDivs(slideIndex += n);
}

function showDivs(n) {
  var i;
  var x = document.getElementsByClassName("mySlides");
  if (n > x.length) {slideIndex = 1}
  if (n < 1) {slideIndex = x.length}
  for (i = 0; i < x.length; i++) {
    x[i].style.display = "none";  
  }
  x[slideIndex-1].style.display = "block";  
}
</script>

In [ ]:
#ignore
完成 rendering gallery

<br>
<br>

從漫威電影到可愛貓咪，從自然風景到人氣偶像，任何你想得到的圖片都可以進行轉換。

一旦訓練完成，我們就能使用 CartoonGAN 在彈指之間將各式各樣的圖片跟動漫做連結。當然，未來類似的應用的效果會更好，且能幫助一般人創造自己的動漫，也能讓動漫作家更快速地畫出草稿、測試自己的新點子。

!quote
- 如 CartoonGAN 這樣的生成模型能讓我們看到未來更多的可能性。

類似的道理我們在[讓 AI 寫點金庸：如何用 TensorFlow 2.0 及 TensorFlow.js 寫天龍八部](https://leemeng.tw/how-to-generate-interesting-text-with-tensorflow2-and-tensorflow-js.html)也已經聊過。

## 訓練你自己的 CartoonGAN

此文篇幅有限。專注在 CartoonGAN 的實際應用，而非演算法細節是因為：
- 網路上已有不少 GAN 理論及演算法的介紹，但能馬上拿來用的例子還不多
- 比起自己花時間訓練出可以用的 GAN，許多人應該只是想自己產生有趣結果

因此，對訓練模型、推論的實作細節有興趣的讀者還請自行參考 [CartoonGan-tensorflow](https://github.com/mnicnc404/CartoonGan-tensorflow)。

不過基本上只要依照指示裝好開發環境以及準備好圖片，你甚至可以一鍵訓練 CartoonGAN：

!image
- cartoongan/train-demo.gif
- 我們的 python script 在訓練過程中提供了詳盡的訊息，方便你理解訓練過程究竟發生什麼事情

因為所有邏輯皆以 TF2 實作，該 Repo 非常適合想要跟上最新 TensorFlow 發展的讀者：
- 使用 [tf.keras](https://www.tensorflow.org/alpha/guide/keras/overview) 實作自定義的 Layers 以及 GAN
- 使用 [tf.data](https://www.tensorflow.org/alpha/guide/data_performance) 讀取大量圖片並進行前處理
- 自定義訓練步驟並使用 [tf.function](https://youtu.be/Up9CvRLIIIw?list=PLQY2H8rRoyvzoUYI26kHmKSJBedn3SQuB) 加快處理速度
- 完整整合 [TensorBoard](https://www.tensorflow.org/tensorboard/r2/get_started#using_tensorboard_with_other_methods
) 以顯示生成圖片與 metrics

!image
- cartoongan/tensorboard-demo.jpg
- 直接在 TensorBoard 上監控 metrics 及生成圖片在訓練 GAN 時非常有效率（圖為我們實際的訓練結果）

跟訓練一個[簡單分類器](https://www.tensorflow.org/tutorials/keras/basic_classification)比起來，訓練一個 GAN 的難度可說是完全不同級別。但我們盡量讓[專案](https://github.com/mnicnc404/CartoonGan-tensorflow)裡頭的程式碼邏輯簡單易懂，希望能讓更多人入門 TensorFlow 2.0 以及 GAN 的訓練。

你可以前往 [CartoonGan-tensorflow](https://github.com/mnicnc404/CartoonGan-tensorflow) 查看所有細節。

## 推薦的 GAN 學習資源

近年以深度學習為基礎的生成模型（Generative Models）領域蓬勃發展，其中最亮眼的發展之一當屬對抗生成網路了。而本文的 CartoonGAN 也是[眾多 GANs](https://github.com/hindupuravinash/the-gan-zoo) 裡頭的其中一個小夥子。

!image
- cartoongan/tf-dcgan.png
- 對抗生成網路 GAN 實際上由兩個獨立的神經網路相互「對抗」
- https://www.tensorflow.org/alpha/tutorials/generative/dcgan

對 GAN 的理論 & 基礎知識有興趣的讀者，我推薦以下的學習資源：

- [李宏毅教授在 Youtube 上的 GAN 教學影片](https://www.youtube.com/watch?v=DQNNMiAP5lw&list=PLJV_el3uVTsMq6JEFPW35BCiOQTsoqwNw)
- [MIT 6.S191 的 Deep Generative Models](https://www.youtube.com/watch?v=yFBFl1cLYx8&index=1&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI)
- [GAN Lab 讓你在瀏覽器上訓練並學習 GAN](https://poloclub.github.io/ganlab/)
- [Andrej Karpathy 的簡單 GAN Demo](https://cs.stanford.edu/people/karpathy/gan/)
- [TensorFlow 官方教學帶你生成 MNIST](https://www.tensorflow.org/alpha/tutorials/generative/dcgan)
- [機器之心講解 CartoonGAN 運作原理](https://www.jiqizhixin.com/articles/CVPR2018-CartoonGAN)
- [Synced: Reproducing Japanese Anime Styles With CartoonGAN AI](https://medium.com/syncedreview/reproducing-japanese-anime-styles-with-cartoongan-ai-cf30d583736e)

!image
- cartoongan/mit-gan.png
- MIT 6.S191 的 Deep Generative Models 適合入門
- https://www.youtube.com/watch?v=yFBFl1cLYx8&index=1&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI

更多學習資源則請參閱[由淺入深的深度學習資源整理](https://leemeng.tw/deep-learning-resources.html)。另外如果你有其他推薦的學習資源，還請不吝留言與我及其他讀者分享，謝謝！

## 結語

CartoonGAN 是我第一次嘗試用 [TensorFlow 2.0](https://www.tensorflow.org/alpha) 以及 [TensorFlow.js](https://www.tensorflow.org/js) 實作的生成對抗網路，從過程中學到了不少東西。感謝大神 [mnicnc404](https://github.com/mnicnc404) 強大的實作支援以及許多寶貴見解，獲益良多。

希望透過這篇文章以及專案能讓更多人對強大的 GAN 以及生成模型產生興趣，並進一步探索相關知識，嘗試更多可能性，做出更多有趣的 AI 應用：）

In [ ]:
#ignore
tfjs setup in next markdown cell

<script src="{static}tfjs-apps/cartoongan/dist/tf.js"></script>
<script src="{static}tfjs-apps/cartoongan/cartoongan.js"></script>

In [ ]:
#ignore
tfjs 引入完畢